<a href="https://colab.research.google.com/github/gilsalvans/DSSG_SS2021/blob/main/Donike_Salvans_DSSG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assessing bias and fairness in the income Adult dataset: Study case for Gender and Race 
This notebook imports the already predicted Adult dataset and assesses bias and fairness using Aequitas and Google What-if for two cases: gender and race.

In [1]:
#imports
import google.colab
import pandas as pd
import numpy as np
!pip install aequitas
import aequitas
!pip install --upgrade witwidget

     |████████████████████████████████| 2.2MB 4.1MB/s 
     |████████████████████████████████| 163kB 20.5MB/s 
     |████████████████████████████████| 92kB 7.3MB/s 
     |████████████████████████████████| 645kB 20.2MB/s 
     |████████████████████████████████| 102kB 8.4MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 460kB 25.9MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 2.6MB 29.7MB/s 
  Created wheel for markdown2: filename=markdown2-2.3.5-cp37-none-any.whl size=33329 sha256=c742eb8593169ddb5ec1e8139617472b33237edca8dc6c2aad0966b2b6d30700
  Stored in directory: /root/.cache/pip/wheels/07/99/0b/89995aca5b1253b89caa7de4041fabea6fba469f42da697664
  Created wheel for xhtml2pdf: filename=xhtml2pdf-0.2.2-cp37-none-any.whl size=230266 sha256=cb64756062be31dcfc37c755f6f21c754eca3383625b0b56f5ea57f3942cbd2b
  Stored in directory: /root/.cache/pip/wheels/37/e9/d9/8ef244092a2b3b004ab197f5c7

In [3]:
# Reading data
# code reused from : https://colab.research.google.com/github/pair-code/what-if-tool/blob/master/WIT_Model_Comparison.ipynb#scrollTo=nu398ARdeuxe
csv_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

# set columns for the csv
csv_columns = [
  "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital-Status",
  "Occupation", "Relationship", "Race", "Sex", "Capital-Gain", "Capital-Loss",
  "Hours-per-week", "Country", "Over-50K"]
# Read the according dataset 
df = pd.read_csv(csv_path, names=csv_columns, skipinitialspace=True)

df

,Age,Workclass,fnlwgt,Education,Education-Num,Marital-Status,Occupation,Relationship,Race,Sex,Capital-Gain,Capital-Loss,Hours-per-week,Country,Over-50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
